![lyon2 geonum](https://perso.liris.cnrs.fr/lmoncla/GEONUM/fig/logos.png)

# 2F2 – Gestion et traitement des données spatio-temporelles


## Tutoriel : Analyse des données des disponibilités des stations Vélo'v de la Métropole de Lyon


# Partie 1 : Exploration de données

Le cours a été réalisé par Ludovic Moncla - Il met à disposition sur sa plateforme des données du Grand Lyon qui ont été retravaillées.

L'objectif de ce tutoriel est d'appréhender la problématique d'analyse de données spatio-temporelles grâce à l'utilisation de librairies Python.
Pour cela nous allons travailler sur un cas d'étude visant la visualisation et le traitement des données de disponibilités des stations Vélo'v de la Métropole de Lyon. 

Les données que nous allons utilisées proviennent de la [plateforme data du Grand Lyon](https://data.grandlyon.com). 


Dans le cadre de ce TP,  vous avez à votre disposition l'ensemble des données pour la période du 7 octobre 2020 au 31 janvier 2021.


Les objectifs de ce tutoriel sont les suivants : 

* Récupérer le jeu de données, analyser sa structure et le charger dans un dataframe
* Explorer et visualiser les données
* Analyser les données : requêter le jeu de données pour générer des graphiques, des cartes et des cartes animées.






## 1. Configurer l'environnement



### 1.1 Importer les librairies

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import geopandas
import wget

## 2. Récupération du jeu de données

Pour palier la limite des 7 jours de disponibilité sur le site du Grand Lyon, Ludovic Moncla a développé un script qui récupère et stocke automatiquement les données chaque jour. Vous aurez ainsi accès aux données pour l'ensemble de l'année 2021. 
Les données sont au format CSV (plus simple à charger dans un dataframe qui le format JSON d'origine). Nous verrons la transformation du format de données lors de la dernière séance.

* Télécharger les archives contenant les données
1. data-stations.zip
2. data-bikes.zip

Ces 2 archives contiennent chacune un fichier CSV contenant respectivement la liste des stations vélov (et leur localisation) et la liste des disponibilités de chaque station par tranche de 30 minutes.


In [ ]:
## On télécharge l'archive contenant la liste des stations
wget.download("https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-stations.zip",out="../data/")
    
## On télécharge l'archive contenant la liste des disponibilité des stations par tranche de 5 minutes
wget.download("https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-bikes.zip",out="../data")

### 2.1. Chargement des données

Dans ce tutoriel nous n'allons pas utiliser de SGBD. L'objectif est de charger l'ensemble des données en mémoire dans une structure Python et de l'interroger directement. 

On distingue deux types de données :
1. les stations vélo'v (id station, latitude, longitude),
2. leurs historiques (id station, année, mois, jour, heure, minute, date, vélos disponibles, places disponibles).

Pour manipuler ces données nous allons utiliser les [dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) de la librairie Pandas.

Pandas est une librairie Python spécialisée dans l'analyse et la manipulation de données. Elle fourni en particulier un objet de type 'dataframe' qui permet de réaliser des opérations de prétraitement et de filtrage que nous utiliserons pour requêter les données.

Les premiers objectifs sont les suivants :

1. Stocker dans un premier dataframe la liste des stations velo'v et leurs coordonnées latitude / longitude associées.
2. Stocker dans un second dataframe pour chaque station et chaque pas de temps les données suivantes : 
    * id de la station
    * année
    * mois
    * jour
    * heure
    * minute
    * date complète (format d'origine)
    * nombre de vélos disponibles
    * nombre de places libres
    * nombre de départs des 30 dernières minutes
    * nombre d'arrivées dess 30 dernières minutes


Pour charger les données il suffit d'utiliser la méthode [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv) de la librairie `Pandas`. Elle prend en paramètre le chemin du fichier que l'on souhaite charger. Ce fichier peut être de 2 formats, soit directement un fichier CSV, soit un fichier ZIP contenant un CSV. Dans notre, cas il est donc inutile de dézipper les archives téléchargées précédemment.


In [ ]:
## On charge les données des stations dans un dataframe
df_stations = pd.read_csv('../data/data-stations.zip')

## On crée maintenant le dataframe avec les données d'historique
df_bikes = pd.read_csv('../data/data-bikes.zip')

In [ ]:
## On vérifie le type de notre variable
type(df_stations)

In [ ]:
## On affiche la liste des colonnes
df_stations.columns

In [ ]:
## On affiche les premières lignes
df_stations.head()

* Combien y a-t-il de stations velo'v ?

In [ ]:
## On affiche la taille du dataframe
## La méthode shape retourne les dimensions (lignes / colonnes)
print(df_stations.shape)

## La fonction len() retourne le nombre de ligne
print(len(df_stations))

In [ ]:
## On affiche les premières lignes
df_bikes.head()

### 2.2. Premier apercu des données d'historique

In [ ]:
## On affiche les information sur les données
df_bikes.info()

In [ ]:
# Réduction de la taille en mémoire

## on transforme le type des colonnes en entier ou float lorsque cela est nécessaire
df_bikes.bikes = df_bikes.bikes.apply(lambda x: int(float(x)))
df_bikes.bike_stands = df_bikes.bike_stands.apply(lambda x: np.int32(float(x)))

df_bikes['year'] = df_bikes['year'].astype('int16')
df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']] = df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']].astype('int8')


In [ ]:
## On affiche les information sur les données
df_bikes.info()

In [ ]:
## Description des données
df_bikes.describe()

In [ ]:
## On affiche 5 lignes sélectionnées de manière aléatoire
df_bikes.sample(5)

### 2.3. Manipulation d'un dataframe

In [ ]:
## Accès à une colonne
df_bikes['time']

In [ ]:
## Accès à une colonne (autre manière en utilisant le .)
df_bikes.time

In [ ]:
## Accès à un ensemble de colonnes
df_bikes[['time', 'bikes']]

In [ ]:
## Récupérer les valeurs d'un ensemble de colonnes
df_bikes[['time', 'bikes']].values

Une colonne (ou variable) est un vecteur de données (Series dans la terminologie de la librarie Pandas).

In [ ]:
## Affichage des premières valeurs d'une seule colonne
df_bikes['time'].head()

In [ ]:
## Affichage des dernières valeurs de la colonne
df_bikes['time'].tail()

In [218]:
## On trie les valeurs d'une colonne de manière croissante
df_bikes['time'].sort_values()

0          2021-01-01 00:00:00+00:00
1363382    2021-01-01 00:00:00+00:00
2467362    2021-01-01 00:00:00+00:00
2703137    2021-01-01 00:00:00+00:00
3349345    2021-01-01 00:00:00+00:00
                     ...            
2398034    2022-01-02 00:00:00+00:00
2398035    2022-01-02 00:00:00+00:00
7503119    2022-01-02 00:00:00+00:00
2328699    2022-01-02 00:00:00+00:00
7800251    2022-01-02 00:00:00+00:00
Name: time, Length: 7800252, dtype: object

In [219]:
## Le tri peut également être généralisé aux DataFrame
## Tri du jeu de données selon l'id de la station et la date
df_bikes.sort_values(by=['id_velov', 'time'])

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7800247,velov-9052,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,9,0,0
7800248,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800249,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800250,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1


In [220]:
## Alternative permettant de remettre l'index des lignes à zéro
df_bikes = df_bikes.sort_values(by=['id_velov', 'time']).reset_index(drop=True)
df_bikes



,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7800247,velov-9052,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,9,0,0
7800248,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800249,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800250,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1


In [221]:
## Comptage des valeurs
df_bikes['id_velov'].value_counts()

id_velov
velov-1013     23392
velov-1005     23389
velov-1003     23388
velov-1001     23388
velov-1016     23388
               ...  
velov-10049    17991
velov-7012     15484
velov-9014      9342
velov-2023      5366
velov-3001       336
Name: count, Length: 343, dtype: int64

In [222]:
## une colonne étant un vecteur il est possible d'utiliser des indices pour accèder aux éléments
## Affichage de la première valeur de la colonne time
df_bikes['time'][0]

'2021-01-01 00:00:00+00:00'

In [223]:
## Affichage des 3 premières valeurs de la colonne time
df_bikes['time'][0:3]

0    2021-01-01 00:00:00+00:00
1    2021-01-01 00:30:00+00:00
2    2021-01-01 01:00:00+00:00
Name: time, dtype: object

#### 2.3.1 Itérations sur les colonnes (variables)

Les itérations sur les variables peuvent se faire via une boucle, ou via l'utilisation de fonctions callback appelée à l'aide d'une fonction `.apply()`.


In [224]:
## Boucler sur l'ensemble des colonne pour afficher leur nom et leur type
for col in df_bikes.columns:
    print(col, ": ", df_bikes[col].dtype)

id_velov :  object
year :  int16
month :  int8
day :  int8
hour :  int8
minute :  int8
time :  object
bikes :  int8
bike_stands :  int8
departure30min :  int8
arrival30min :  int8


#### 2.3.2 Itérations sur les lignes (\*\*déconseillé dans le cas des grands dataframe**)


Il est possible de parcourir les lignes d'un dataframe, mais attention, l'itération sur un dataframe est lent. Mieux vaut utiliser des opérations vectorielles ! Si on ne peut pas, on préfére utiliser une fonction callback appelée à l'aide d'une fonction `.apply()`.

Remarque : on ne peut pas modifier un dataframe sur lequel on boucle.



In [225]:
## Pour l'exemple, on itère sur le dataframe des stations (car celui de l'historique est trop grand)
for index, row in df_stations.iterrows():
    print('ID :', row.id_velov, '\t lat :', row.latitude,'\t lng :', row.longitude)


ID : velov-10056 	 lat : 45.779112 	 lng : 4.871952
ID : velov-9013 	 lat : 45.787384 	 lng : 4.814374
ID : velov-5044 	 lat : 45.759797 	 lng : 4.796627
ID : velov-9014 	 lat : 45.783341 	 lng : 4.811433
ID : velov-10058 	 lat : 45.767126 	 lng : 4.89215
ID : velov-9022 	 lat : 45.778554 	 lng : 4.80704
ID : velov-10060 	 lat : 45.758788 	 lng : 4.878976
ID : velov-2009 	 lat : 45.739608 	 lng : 4.815074
ID : velov-10034 	 lat : 45.761788 	 lng : 4.886157
ID : velov-2036 	 lat : 45.75947 	 lng : 4.830145
ID : velov-6006 	 lat : 45.77332 	 lng : 4.85183
ID : velov-1016 	 lat : 45.765949 	 lng : 4.831107
ID : velov-6001 	 lat : 45.766855 	 lng : 4.858974
ID : velov-9049 	 lat : 45.770134 	 lng : 4.805158
ID : velov-5026 	 lat : 45.763376 	 lng : 4.82904
ID : velov-6008 	 lat : 45.768412 	 lng : 4.859071
ID : velov-9040 	 lat : 45.771006 	 lng : 4.807613
ID : velov-10005 	 lat : 45.779702 	 lng : 4.859991
ID : velov-2038 	 lat : 45.75308 	 lng : 4.829649
ID : velov-1020 	 lat : 45.766806

#### 2.3.3 Accès indicé aux données d'un DataFrame

On peut accéder aux valeurs du DataFrame via des indices ou plages d'indices. 
La structure se comporte alors comme une matrice. La cellule en haut à gauche est de coordonnées (0,0).
Il y a différentes manières de le faire, l'utilisation de `.iloc[,]` constitue une des solutions les plus simples.
Rappel : la méthode `shape()` permet d'obtenir les dimensions (lignes et colonnes) du DataFrame.


In [226]:
## Accès à la valeur située en (0,0) (première ligne, première colonne)
df_bikes.iloc[0,0]

'velov-10001'

In [227]:
## Valeur située en dernière ligne, première colonne
## Utilisation de l'indiçage négatif
df_bikes.iloc[-1,0]

'velov-9052'

In [228]:
## Alternative avec shape, valeur située en dernière ligne, première colonne
## shape[0] renvoie le nombre de lignes (première dimension)
## il faut réduire de -1 parce le premier indice est égal à 0 sinon on déborde
df_bikes.iloc[df_bikes.shape[0]-1,0]

'velov-9052'

In [229]:
## Affichage des 5 premières valeurs de toutes les colonnes
## lignes => 0:5 (0 à 5 [non inclus])
## colonnes = : (toutes les colonnes)
df_bikes.iloc[0:5,:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0


In [230]:
## Avec l'indiçage négatif, on peut facilement accéder aux 5 dernières lignes
df_bikes.iloc[-5:,:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
7800247,velov-9052,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,9,0,0
7800248,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800249,velov-9052,2022,1,1,23,30,2022-01-01 23:30:00+00:00,3,9,0,0
7800250,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1
7800251,velov-9052,2022,1,2,0,0,2022-01-02 00:00:00+00:00,4,8,0,1


In [231]:
## 5 premières lignes et colonnes 0, 6, 7 et 8
## on a une liste d'indices en colonne
df_bikes.iloc[0:5,[0,6,7,8]]

,id_velov,time,bikes,bike_stands
0,velov-10001,2021-01-01 00:00:00+00:00,8,22
1,velov-10001,2021-01-01 00:30:00+00:00,8,22
2,velov-10001,2021-01-01 01:00:00+00:00,7,23
3,velov-10001,2021-01-01 01:30:00+00:00,7,23
4,velov-10001,2021-01-01 02:00:00+00:00,7,23


#### 2.3.4 Filtrage avec des conditions - Les requêtes

Nous pouvons isoler les sous-ensembles d'observations répondant à des critères définis sur les champs. Nous utiliserons préférentiellement la méthode `.loc[,]` dans ce cadre.

In [232]:
## Liste des données d'historique pour la station 'velov-10001'
df_bikes.loc[df_bikes['id_velov']=="velov-10001",:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22848,velov-10001,2022,1,1,23,0,2022-01-01 23:00:00+00:00,7,21,0,0
22849,velov-10001,2022,1,1,23,30,2022-01-01 23:30:00+00:00,7,21,0,0
22850,velov-10001,2022,1,1,23,30,2022-01-01 23:30:00+00:00,7,21,0,0
22851,velov-10001,2022,1,2,0,0,2022-01-02 00:00:00+00:00,7,21,0,0


In [233]:
## Pour un ensemble de valeurs de la même variable, on utilise la méthode isin()
df_bikes.loc[df_bikes['id_velov'].isin(['velov-10001','velov-10002']),:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
0,velov-10001,2021,1,1,0,0,2021-01-01 00:00:00+00:00,8,22,0,0
1,velov-10001,2021,1,1,0,30,2021-01-01 00:30:00+00:00,8,22,0,0
2,velov-10001,2021,1,1,1,0,2021-01-01 01:00:00+00:00,7,23,1,0
3,velov-10001,2021,1,1,1,30,2021-01-01 01:30:00+00:00,7,23,0,0
4,velov-10001,2021,1,1,2,0,2021-01-01 02:00:00+00:00,7,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...
41215,velov-10002,2022,1,1,23,0,2022-01-01 23:00:00+00:00,3,52,0,0
41216,velov-10002,2022,1,1,23,30,2022-01-01 23:30:00+00:00,5,50,0,2
41217,velov-10002,2022,1,1,23,30,2022-01-01 23:30:00+00:00,5,50,0,1
41218,velov-10002,2022,1,2,0,0,2022-01-02 00:00:00+00:00,5,50,0,0


Des opérateurs logiques permettent de combiner les conditions. 
On utilise respectivement : & pour ET, | pour OU, et ~ pour la négation.

In [234]:
## Liste des données pour la station 'velov-10001' et hour = 8
df_bikes.loc[(df_bikes['id_velov']=="velov-10001") & (df_bikes['hour'] == 8),:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
16,velov-10001,2021,1,1,8,0,2021-01-01 08:00:00+00:00,7,23,0,0
17,velov-10001,2021,1,1,8,30,2021-01-01 08:30:00+00:00,7,23,0,0
65,velov-10001,2021,1,2,8,0,2021-01-02 08:00:00+00:00,5,25,0,0
66,velov-10001,2021,1,2,8,30,2021-01-02 08:30:00+00:00,5,25,0,0
114,velov-10001,2021,1,3,8,0,2021-01-03 08:00:00+00:00,8,22,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22694,velov-10001,2021,12,31,8,30,2021-12-31 08:30:00+00:00,6,24,0,0
22787,velov-10001,2022,1,1,8,0,2022-01-01 08:00:00+00:00,7,22,0,0
22788,velov-10001,2022,1,1,8,0,2022-01-01 08:00:00+00:00,7,22,0,0
22789,velov-10001,2022,1,1,8,30,2022-01-01 08:30:00+00:00,7,22,0,0


In [235]:
## Liste des données datant d'après juillet
df_bikes.loc[(df_bikes['month'] > 7),:]

,id_velov,year,month,day,hour,minute,time,bikes,bike_stands,departure30min,arrival30min
14839,velov-10001,2021,8,1,0,0,2021-08-01 00:00:00+00:00,15,15,0,1
14840,velov-10001,2021,8,1,0,0,2021-08-01 00:00:00+00:00,15,15,0,1
14841,velov-10001,2021,8,1,0,30,2021-08-01 00:30:00+00:00,15,15,0,0
14842,velov-10001,2021,8,1,1,0,2021-08-01 01:00:00+00:00,15,15,0,0
14843,velov-10001,2021,8,1,1,30,2021-08-01 01:30:00+00:00,15,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7800149,velov-9052,2021,12,31,22,30,2021-12-31 22:30:00+00:00,6,6,0,0
7800150,velov-9052,2021,12,31,23,0,2021-12-31 23:00:00+00:00,6,6,0,0
7800151,velov-9052,2021,12,31,23,0,2021-12-31 23:00:00+00:00,6,6,0,0
7800152,velov-9052,2021,12,31,23,30,2021-12-31 23:30:00+00:00,6,6,0,0


In [236]:
#on peut n'afficher qu'une partie des colonnes
#on définit la projection dans une liste
colonnes = ['id_velov','time','bikes','bike_stands']
#que l'on utilise en paramètre dans .loc[]
#pour la même restruction que précédemment
df_bikes.loc[(df_bikes['month'] > 7),colonnes]

,id_velov,time,bikes,bike_stands
14839,velov-10001,2021-08-01 00:00:00+00:00,15,15
14840,velov-10001,2021-08-01 00:00:00+00:00,15,15
14841,velov-10001,2021-08-01 00:30:00+00:00,15,15
14842,velov-10001,2021-08-01 01:00:00+00:00,15,15
14843,velov-10001,2021-08-01 01:30:00+00:00,15,15
...,...,...,...,...
7800149,velov-9052,2021-12-31 22:30:00+00:00,6,6
7800150,velov-9052,2021-12-31 23:00:00+00:00,6,6
7800151,velov-9052,2021-12-31 23:00:00+00:00,6,6
7800152,velov-9052,2021-12-31 23:30:00+00:00,6,6


* Combien de stations vélo'v on eu plus de 5 départs le 20 septembre 2021 à 12h00 ? 

In [237]:
****

SyntaxError: invalid syntax (749935734.py, line 1)

#### 2.3.5 Regroupement des variables

L'utilisation de `groupby()` permet d'accéder aux sous-DataFrame associés à chaque item de la variable de regroupement. Il est dès lors possible d'appliquer explicitement d'autres traitements sur ces sous-ensembles de données.

In [ ]:
#regroupement des données selon le l'id de la station
g = df_bikes.groupby(['id_velov'])

g.size()

In [ ]:
#calculer la dimension du sous-DataFrame associé à la station 'velov_10001'
g.get_group('velov-10001').shape

### 2.4. Visualisation des localisations des stations

Maintenant que vous avez chargé les données en mémoire et vue comment manipuler un `DataFrame`, vous allez produire votre première carte.



#### 2.4.1 Utilisation des librairies GeoPandas et Plotly 

La librairie [GeoPandas](https://geopandas.org/) est conçu pour faciliter la manipulation de données spatiales. La particularité de GeoPandas est qu'elle permet de manipuler les données spatiales comme s'il s'agissait de données traditionnelles. 

Par rapport à un `DataFrame` standard, un `GeoDataFrame`, comporte une colonne supplémentaire: `geometry`. Comme dans un SGBD spatial, cette colonne permet de stocker les contours (la géométrie) d'un objet géographique. Un objet `GeoDataFrame` hérite des propriétés d'un `DataFrame` mais propose des méthodes adaptées au traitement des données spatiales.

Ainsi en plus des manipulations déjà possible avec pandas, on pourra manipulation la dimension spatiale : 
- calculer des distances et des surfaces,
- agréger rapidement des zonages (regrouper les départements en région par exemple),
- rechercher une zone à partir des coordonnées d'un point,
- convertir les données dans différents systèmes de projection,
- faire une carte.

Pour le moment on s'intèresse au dernier point afin de produire une carte des stations Velo'v.

![stations velov avec GeoPandas](https://perso.liris.cnrs.fr/lmoncla/GEONUM/fig/geopandas_stations.png)

* Affichez les stations vélo'v sur une carte. Utiliser la librairie [GeoPandas](https://geopandas.org/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py). Vous devez obtenir le résultat ci-dessus.

In [ ]:
## On transforme le dataframe des stations en geodataframe (https://geopandas.org/gallery/create_geopandas_from_pandas.html#sphx-glr-gallery-create-geopandas-from-pandas-py)
gdf_stations = ****


In [ ]:
## On affiche les premières lignes du GeoDataFrame pour vérifier l'existance de la colonne géométrie
gdf_stations.head()

In [ ]:
## On affiche directement les données du geodataframe sur une carte 
## avec la méthode scatter_mapbox() de la librairie plotly.express:
fig = px.scatter_mapbox(gdf_stations,
                        lat=****,
                        lon=****,
                        hover_name="id_velov",
                        zoom=12, mapbox_style="carto-positron")

## On supprime les marges autour de la carte
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

## On affiche la carte
fig.show()

## 3. Préparation des données

### 3.1. Ajout d'information

Avant de pouvoir analyser les données d'historique, on souhaite ajouter quelques informations. Par exemple, le jeu de données initial ne fournit pas directement les trajets (départs / arrivées) des utilisateurs mais seulement le nombre de vélos ou de places disponibles à un instant t (par tranche de 5 minutes). Pour faire une analyse de la fréquentation ou des zones de départs et d'arrivées en fonction du moment de la journée ou de la semaine j'ai calculé les départs et arrivées par tranches de 30min.

A partir des tranches de 30 min nous pouvons par exemple inférer le nombre quotidien.


In [ ]:
## On commence par faire une copie de notre DataFrame, pour pouvoir revenir aux données initiales si besoin
df_sampled = df_bikes.copy()

#### 3.1.1 Calcul du nombre d'arrivées et de départs quotidiens


On peut faire des calculs directement en groupant les lignes grâce à la méthode `groupby()`.

Quelles colonnes faut-il regrouper pour pouvoir calculer les départs et arrivées quotidiens ?

In [ ]:
df_sampled.head()

In [ ]:
## La méthode 'transform' permet d'appliquer un calcul au dataframe d'origine (non groupé). 
## Dans notre cas, on souhaite effectuer une somme sur les colonnes departure30min et arrival30min.

## Compléter la liste des colonnes
df_sampled["daily_departure"] = df_sampled.groupby([****])['departure30min'].transform('sum')
df_sampled["daily_arrival"] = df_sampled.groupby([****])['arrival30min'].transform('sum')


In [ ]:
## On affiche 15 lignes aléatoirement pour visualiser le résultat
df_sampled.sample(15)

#### 3.1.2 Distinction semaine / weekend

Afin d'analyser les données on souhaite pouvoir distinguer les jours de la semaine des jours de weekend, pour cela nous devons préparer les données afin d'identifier les jours de weekend.

1. On défini une fonction qui retourne vrai lorsque la date est un jour de la semaine et faux lorsque c'est le weekend
2. On applique cette fonction sur chaque ligne de notre dataframe

In [ ]:
## La fonction weekDay, prend 3 paramètres : l'année, le mois et le jour
def weekDay(year, month, day):
    ## Cette méthode retourne vrai (True) si la date correspond à un jour de la semaine, faux (False) sinon
    ## On utilise la fonction datetime() et la méthode weekday()
    ## https://docs.python.org/fr/3/library/datetime.html#datetime.datetime
    
    ****

## On vectorise la fonction afin de l'appliquer de manière efficace (en terme de temps de calcul) sur le dataframe
isWeekDay = np.vectorize(weekDay)

In [ ]:
## On ajoute une nouvelle colonne à partir du résultat de la fonction appliquée sur l'ensemble des lignes du dataframe
df_sampled['IsWeekday'] = isWeekDay(****)


In [ ]:
## on affiche un échantillon du dataframe
df_sampled.sample(10)

* Créer une nouvelle colonne `day_of_week` qui contient le jour de la semaine (0 pour lundi, 1 pour mardi, etc.)

In [ ]:
df_sampled['day_of_week'] = df_sampled.apply(lambda row: datetime(row['year'], row['month'], row['day']).weekday(), axis=1)

In [ ]:
## on affiche un échantillon du dataframe
df_sampled.sample(10)

## 4. Sauvegarde du jeu de données préparé

Afin de pouvoir réutiliser le jeu de données sans refaire tous les traitements on l'enregistre dans un fichier CSV.

Utiliser la méthode [to_csv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) de la librairie Pandas pour enregistrer le dataframe modifié dans un fichier.

In [ ]:
## On enregistre le dataframe modifié

compression_opts = dict(method='zip', archive_name='data-bikes-2.csv')  
df_sampled.to_csv('../data/data-bikes-2.zip', index=False, compression=compression_opts)